[View in Colaboratory](https://colab.research.google.com/github/santhavathi/Deep-Learning-MLBLR/blob/master/DenseNet_CIFAR10_Final.ipynb)

In [0]:
import numpy as np

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 28s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
batch_size=64
num_filter = 12
dropout_rate = 0.2
l = 20

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (


concatenate_50 (Concatenate)    (None, 8, 8, 66)     0           concatenate_49[0][0]             
                                                                 dropout_52[0][0]                 
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 8, 8, 66)     264         concatenate_50[0][0]             
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 8, 8, 66)     0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 8, 8, 6)      3564        activation_53[0][0]              
__________________________________________________________________________________________________
dropout_53 (Dropout)            (None, 8, 8, 6)      0           conv2d_54[0][0]                  
_________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
#Took approximately 6.5hrs to train, smaller batch size more time
#Total 50 epochs, 5 for loop iterartions each 10 epochs.
#Below is the output of the 5th run of 10 epochs
from google.colab import files

for i in range(1,6):
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

  file_name = 'DenseNet_CIFAR10_' + str(i) + '.h5'

  # Save the trained weights in to .h5 format
  model.save(file_name)
  #model.save_weights("DenseNet_CIFAR10.h5")
  print("Saved model to disk - {0}".format(file_name))
  !ls

  files.download(file_name)
  model.load_weights(file_name)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
43776/50000 [=========================>....] - ETA: 56s - loss: 0.3262 - acc: 0.8833

50000/50000 [==============================] - 471s 9ms/step - loss: 0.3274 - acc: 0.8824 - val_loss: 0.6568 - val_acc: 0.8131
Epoch 2/10
26240/50000 [==============>...............] - ETA: 3:33 - loss: 0.3236 - acc: 0.8862

50000/50000 [==============================] - 470s 9ms/step - loss: 0.3231 - acc: 0.8870 - val_loss: 0.6884 - val_acc: 0.8126
Epoch 3/10
19520/50000 [==========>...................] - ETA: 4:32 - loss: 0.3243 - acc: 0.8846

50000/50000 [==============================] - 468s 9ms/step - loss: 0.3233 - acc: 0.8846 - val_loss: 0.6458 - val_acc: 0.8161
Epoch 4/10
16960/50000 [=========>....................] - ETA: 4:57 - loss: 0.3222 - acc: 0.8866

50000/50000 [==============================] - 471s 9ms/step - loss: 0.3225 - acc: 0.8858 - val_loss: 1.0980 - val_acc: 0.7504
Epoch 5/10
16000/50000 [========>.....................] - ETA: 5:00 - loss: 0.3114 - acc: 0.8896

...

50000/50000 [==============================] - 471s 9ms/step - loss: 0.3154 - acc: 0.8881 - val_loss: 0.6416 - val_acc: 0.8240
Epoch 8/10
15488/50000 [========>.....................] - ETA: 5:09 - loss: 0.2980 - acc: 0.8952

50000/50000 [==============================] - 469s 9ms/step - loss: 0.3131 - acc: 0.8893 - val_loss: 0.5657 - val_acc: 0.8385
Epoch 9/10
15424/50000 [========>.....................] - ETA: 5:06 - loss: 0.3041 - acc: 0.8917

50000/50000 [==============================] - 468s 9ms/step - loss: 0.3088 - acc: 0.8905 - val_loss: 0.7012 - val_acc: 0.8049
Epoch 10/10
15360/50000 [========>.....................] - ETA: 5:10 - loss: 0.2960 - acc: 0.8952

50000/50000 [==============================] - 467s 9ms/step - loss: 0.3047 - acc: 0.8916 - val_loss: 0.6198 - val_acc: 0.8288
Saved model to disk - DenseNet_CIFAR10_10.h5
datalab			DenseNet_CIFAR10_6.h5	   DenseNet_CIFAR10_8.h5
DenseNet_CIFAR10_10.h5	DenseNet_CIFAR10_7 (1).h5  DenseNet_CIFAR10_9.h5
DenseNet_CIFAR10_5.h5	DenseNet_CIFAR10_7.h5


MessageError: ignored

In [0]:
files.download(file_name)
model.load_weights(file_name)

In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print(score)

10000/10000 [==============================] - 29s 3ms/step
[0.6198364387512207, 0.8288]
